# iPhone case (Neural Network training)

*Sofia Shchipinskaya*

### Short plan of training part

1. Transforming the data
2. Installing ResNet18
3. Building the model (defining parameters, ect.)
4. Running the model
5. Converting the model
6. Checking the model performance

In [1]:
# importing libraries
import os
import glob
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import resnet50, inception_v3, vgg16
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.optimizers import Adam
import keras
from keras import backend as K
from keras.models import load_model
import tensorflow as tf
from sklearn import metrics
from keras.callbacks import LearningRateScheduler
import math
from classification_models.resnet import ResNet18
from keras.models import load_model
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from tensorflow.contrib import lite
from sklearn.metrics import average_precision_score

Using TensorFlow backend.


### Transformation of the data

In [2]:
# creating random transformation
val_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        rescale=1./255,
        fill_mode='nearest')

train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        rescale=1./255,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(
        rescale=1./255,
        fill_mode='nearest')

In [3]:
# creating data loaders
val_generator = val_datagen.flow_from_directory(
        'full dataset/val',  
        target_size=(224, 224),  
        batch_size=64,
        class_mode='binary')  

train_generator = val_datagen.flow_from_directory(
        'full dataset/train',  
        target_size=(224, 224), 
        batch_size=64,
        class_mode='binary')  

test_generator = val_datagen.flow_from_directory(
        'full dataset/test',  
        target_size=(224, 224), 
        batch_size=64,
        shuffle=False,
        class_mode='binary')  

Found 2000 images belonging to 2 classes.
Found 48684 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [4]:
train_generator.class_indices

{'An': 0, 'Ip': 1}

### Installing the neural network and running it

In [5]:
# installing ResNet18
!pip install git+https://github.com/qubvel/classification_models.git

  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-l_xdgcmp
  Stored in directory: /tmp/pip-ephem-wheel-cache-lf4diab6/wheels/de/2b/fd/29a6d33edb8c28bc7d94e95ea1d39c9a218ac500a3cfb1b197
Successfully built image-classifiers


In [6]:
# building the model
model = ResNet18(input_shape=(224,224,3), weights='imagenet', include_top=True)
model.layers.pop()
last = model.layers[-1].output
x = Dense(2, activation="softmax")(last)
model = Model(model.input, x)

Instructions for updating:
Colocations handled automatically by placer.


#### comment
The next step is to define an optimizer. I tried Adam and SGD optimizers, however, in this version of the model i decided to use Adam optimizer.

In [14]:
# defining optimizer
# sgd = optimizers.SGD(lr=0.001, decay=0.0, momentum=0.0, nesterov=False)

In [7]:
# defining hyperparameters for tunning
model.compile(loss='sparse_categorical_crossentropy',
#               optimizer=sgd,
              optimizer='adam',
              metrics=['accuracy'])

# model.compile(loss='sparse_categorical_crossentropy',
#               optimizer=sgd,
# #               optimizer=Adam(lr=0.001),
#               metrics=['acc',f1])
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# defining batch sizes
batch_size=64

In [9]:
# defining sample sizes 
nb_samples_train = len(train_generator.filenames)
nb_samples_val= len(val_generator.filenames)
nb_samples_test = len(test_generator.filenames)

#### comment
I decided to use learing rate schedule, so there are some steps to set it.

In [10]:
# learning rate schedule
def step_decay(epoch):
    initial_lrate=0.001
    drop=0.5
    epochs_drop=10
    lrate=initial_lrate*math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

In [11]:
# learning rate callback
lrate = LearningRateScheduler(step_decay)
callbacks_list = [lrate]

#### comment
The next part is the part of model training. As you can see, I chose to train only 2 epochs. In the previous version of training part I set 100 epochs and trained them all (you can see that file in my repo). Due to techical reasons (poor Internet connection and some problems with Google Cloud GPU) I had no time to train the model (previous model took 24h to train with epoch=100 and batch_size=16). In this version I chose epoch=2 and batch_size=64. So, to upload my model on time, I decided to make quite "fast" calculations.

In [12]:
# training the model
model.fit_generator(
        train_generator,
        steps_per_epoch=nb_samples_train/batch_size,
        epochs=2,
        validation_data=val_generator,
        validation_steps=nb_samples_val/batch_size,
        callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
761/760 [==============================] - 645s 848ms/step - loss: 0.4835 - acc: 0.7777 - val_loss: 0.7287 - val_acc: 0.7315
Epoch 2/2
761/760 [==============================] - 635s 835ms/step - loss: 0.3561 - acc: 0.8365 - val_loss: 0.5692 - val_acc: 0.7740


In [13]:
# saving the model and weights
model.save('my_model2.h5')
model.save_weights('my_weights2.h5')

#### comment
As GitHub doesn't allow to upload files of more than 100Mb, I converted the model to decrease the size.

In [14]:
# Converting keras model to tflite model
converter = lite.TFLiteConverter.from_keras_model_file('my_model2.h5')
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 100 variables.
INFO:tensorflow:Converted 100 variables to const ops.


46791100

### Checking perfomance

In [15]:
#create function for loading and rescaling images
def load_image(img_path):

    img = image.load_img(img_path, target_size=(224, 224))
    img_tensor = image.img_to_array(img)                    
    img_tensor = np.expand_dims(img_tensor, axis=0)         
    img_tensor /= 255.
    
    return img_tensor

In [16]:
# loading converted model, setting input and output
interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [18]:
# storing predictions
pred = []
for subdir, dirs, files in os.walk('full dataset/test'):
    for file in files:
        input_data = load_image(os.path.join(subdir, file))
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        pred.append(output_data[0])

In [20]:
# calculating avarage percision
average_precision = average_precision_score(test_generator.classes, [x[1] for x in pred])
print('Average precision is: {0:0.10f}'.format(average_precision))

Average precision is: 0.9172811585


#### comment
Even if i trained here only 2 epochs, the average precision is quite high (91.72%)